# <center>Apply a bone tumor segmenter on an radiomic exam</center>

In [1]:
# imports
import ipywidgets as widgets
from PIL import Image
import io
import numpy as np

from detectron2.utils.visualizer import ColorMode
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo
from detectron2.data import DatasetCatalog, MetadataCatalog

register_coco_instances(
    "my_dataset_train", {}, "./train.json", ''
)

from IPython.display import HTML
loading = HTML('<img src="img/loading.gif">')

In [2]:
cfg = get_cfg()

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.DATALOADER.SAMPLER_TRAIN = "RepeatFactorTrainingSampler"
cfg.DATALOADER.REPEAT_THRESHOLD = 0.001
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = './model.pth'
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.MODEL.DEVICE= 'cpu'
mal_col = (0, 69, 255)  # red
ben_col = (50, 205, 50)  # green

max_width = 800
max_height = 1200


MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_colors = [ben_col, mal_col]
predictor = DefaultPredictor(cfg)
#cfg.DATASETS.TRAIN

### Upload the desired radiograph

In [3]:
btn_upload = widgets.FileUpload()
box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='50%')

btn_upload

FileUpload(value={}, description='Upload')

(1697, 929)
(800, 929)


In [4]:
def center_crop(im):

    sz = im.size   # Get dimensions
    width, height = sz[0], sz[1]
    print(sz)
    
    new_width = min(width, max_width)
    new_height = min(height, max_height)

    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2

    # Crop the center of the image
    im = im.crop((left, top, right, bottom))
    print(im.size)
    return im

In [5]:
def on_data_change(change):
    out_pl.clear_output()
    with out_pl: display(loading)
    img = Image.open(io.BytesIO(btn_upload.data[-1]))
    img = center_crop(img)
    out_pl.clear_output()
    with out_pl: display(img)

btn_upload.observe(on_data_change, names=['data'])

In [6]:
out_pl = widgets.Output()
out_pl.clear_output()
out_pl

Output()

### Segment the tumor, this takes about 1 min.

In [7]:
btn_run = widgets.Button(description='Segment')
btn_run

Button(description='Segment', style=ButtonStyle())

(1697, 929)
(800, 929)
(929, 800, 3)


C:\Users\Niko\anaconda3\lib\site-packages\detectron2\layers\wrappers.py:240: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:766.)
  return x.nonzero().unbind(1)


In [8]:
def on_click_segment(change):
    out_pl.clear_output()
    with out_pl: display(loading)
    img = Image.open(io.BytesIO(btn_upload.data[0]))
    img = np.array(center_crop(img))[:, :, :3]
    print(img.shape)
    
    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")[:1]
    
    v = Visualizer(
        img[:, :, ::-1],
        metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN[0]),
        scale=1,
        instance_mode=ColorMode.SEGMENTATION,
    )
    v = v.draw_instance_predictions(instances)
    back_img = Image.fromarray(v.get_image()[:, :, ::-1]) 
    
    out_pl.clear_output()
    with out_pl: display(back_img)

btn_run.on_click(on_click_segment)

In [9]:
#!pip install voila
#!jupyter serverexternsion enable voila --sys-prefix

This segmenter is for demonstration purposes only and does not provide any kind of medical approval or advice.
For questions contact nikolas.wilhelm@tum.de